In [1]:
import pandas as pd

df_educacion = pd.read_csv("noticias_educacion_sample.csv")
df_educacion['clase'] = 0
df_politica = pd.read_csv("noticias_politica_sample.csv")
df_politica['clase'] = 1
df_deportes = pd.read_csv("noticias_deportes_sample.csv")
df_deportes['clase'] = 2
df_economia = pd.read_csv("noticias_economia_sample.csv")
df_economia['clase'] = 3
df = pd.concat([df_educacion, df_politica, df_deportes, df_economia]).dropna().reset_index()

In [2]:
df

,index,content,date,headline,description,clase
0,0,Como parte de la política de puertas abiertas ...,2022-02-08T19:12:01.737Z,La CAN abre convocatorias para pasantías en Co...,La Comunidad Andina de Naciones abrió la posib...,0
1,1,"El programa, que cumple 30 años desde su prime...",2022-05-14T18:02:23.629Z,Colfuturo apoyará a 1.526 profesionales colomb...,"Los beneficiarios, en su mayoría, realizaron e...",0
2,2,Estudiar una carrera universitaria en Colombia...,2022-10-19T09:45:01.712Z,¿Cómo estudiar becado en la mejor universidad ...,"Según el ranking de Times Higher Education, la...",0
3,3,Escuche aquí el episodio número 27 de Finanzas...,2021-04-07T17:56:34.238Z,Consejos para financiar con inteligencia sus e...,Si estudiar es uno de sus principales objetivo...,0
4,4,Durante el último año de la carrera universita...,2022-04-02T18:08:22.865Z,Pruebas Saber Pro: el listado de universidades...,Las universidades públicas presentaron preocup...,0
...,...,...,...,...,...,...
1928,495,Colombia sigue aumentando su endeudamiento ext...,2023-02-10T23:08:47.922Z,"Deuda externa de Colombia representó el 52,8% ...",Así lo deja en evidencia el más reciente repor...,3
1929,496,La Agencia de Estados Unidos para el Desarroll...,2022-09-28T17:00:15.603Z,Lanzan convocatoria para apoyar a más de mil o...,La Usaid estará al frente de este proceso que ...,3
1930,497,La inflación es uno de los mayores retos que e...,2023-02-25T03:41:20.639Z,Controlar la inflación no será tan fácil como ...,El aumento en los precios será una constante e...,3
1931,498,23 lugares icónicos de Cúcuta fueron decorados...,2022-12-07T17:16:46.317Z,Reapertura económica en la frontera: artesanas...,Cúcuta prepara la Ruta Navideña luego de haber...,3


In [3]:
df.loc[0, 'content']

'Como parte de la política de puertas abiertas a las nuevas generaciones andinas, el secretario general de la Comunidad Andina de Naciones (CAN), Jorge Hernando Pedraza, anunció la convocatoria al XI programa de pasantías profesionales en el organismo andino “practiCAN”. Según se informó, podrán aplicar estudiantes universitarios que estén cursando el último semestre académico, egresados, técnicos y profesionales recién titulados en un tiempo no mayor a un año de diferentes escuelas y facultades universitarias de Bolivia, Colombia, Ecuador y Perú. “Esta iniciativa busca fortalecer una mayor vinculación de los jóvenes de los países miembros con el proceso andino de integración e impulsar que las nuevas generaciones asuman un rol más activo en la región desarrollando sus capacidades profesionales en las diversas áreas de la Secretaría General”, señaló el jefe del organismo. Podrán ser parte de “practiCAN 2022″, los jóvenes de las carreras Comercio Exterior, Relaciones Internacionales, Ne

In [4]:
df

,index,content,date,headline,description,clase
0,0,Como parte de la política de puertas abiertas ...,2022-02-08T19:12:01.737Z,La CAN abre convocatorias para pasantías en Co...,La Comunidad Andina de Naciones abrió la posib...,0
1,1,"El programa, que cumple 30 años desde su prime...",2022-05-14T18:02:23.629Z,Colfuturo apoyará a 1.526 profesionales colomb...,"Los beneficiarios, en su mayoría, realizaron e...",0
2,2,Estudiar una carrera universitaria en Colombia...,2022-10-19T09:45:01.712Z,¿Cómo estudiar becado en la mejor universidad ...,"Según el ranking de Times Higher Education, la...",0
3,3,Escuche aquí el episodio número 27 de Finanzas...,2021-04-07T17:56:34.238Z,Consejos para financiar con inteligencia sus e...,Si estudiar es uno de sus principales objetivo...,0
4,4,Durante el último año de la carrera universita...,2022-04-02T18:08:22.865Z,Pruebas Saber Pro: el listado de universidades...,Las universidades públicas presentaron preocup...,0
...,...,...,...,...,...,...
1928,495,Colombia sigue aumentando su endeudamiento ext...,2023-02-10T23:08:47.922Z,"Deuda externa de Colombia representó el 52,8% ...",Así lo deja en evidencia el más reciente repor...,3
1929,496,La Agencia de Estados Unidos para el Desarroll...,2022-09-28T17:00:15.603Z,Lanzan convocatoria para apoyar a más de mil o...,La Usaid estará al frente de este proceso que ...,3
1930,497,La inflación es uno de los mayores retos que e...,2023-02-25T03:41:20.639Z,Controlar la inflación no será tan fácil como ...,El aumento en los precios será una constante e...,3
1931,498,23 lugares icónicos de Cúcuta fueron decorados...,2022-12-07T17:16:46.317Z,Reapertura económica en la frontera: artesanas...,Cúcuta prepara la Ruta Navideña luego de haber...,3


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')

# Download the NLTK Spanish stopwords if you haven't already
spanish_stopwords = nltk.corpus.stopwords.words('spanish')

# Initialize the TfidfVectorizer
vectorizer_tfidf = TfidfVectorizer(max_features=2000, stop_words=spanish_stopwords, ngram_range=(1,3), lowercase=True)

df = df[['content', 'clase']].dropna()

# Train the TfidfVectorizer and transform the texts
X_tfidf = vectorizer_tfidf.fit_transform(df['content']).todense()
y = df['clase']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carit\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import xgboost as xgb

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=1)

# train XGBoost model
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

# make predictions on test set
y_pred = model.predict(X_test)

# compute precision, recall, and F1 score for each class
num_classes = len(set(y_test))
precision_scores = []
recall_scores = []
f1_scores = []
for i in range(num_classes):
    class_predicted = [1 if x == i else 0 for x in y_pred]
    class_real = [1 if x == i else 0 for x in y_test]
    precision = precision_score(class_real, class_predicted)
    recall = recall_score(class_real, class_predicted)
    f1 = f1_score(class_real, class_predicted)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

# print results
for i in range(num_classes):
    print("Class {}: Precision: {:.2f}, Recall: {:.2f}, F1 score: {:.2f}".format(i, precision_scores[i], recall_scores[i], f1_scores[i]))

# get feature importances and vocabulary
feature_importances = model.feature_importances_
vocabulary = vectorizer_tfidf.get_feature_names()

# print the top 10 tokens with the largest feature importance
indices = feature_importances.argsort()[::-1]
print("Top 10 tokens with largest feature importance:")
for i in range(10):
    print("{:<15} {:.2f}%".format(vocabulary[indices[i]], feature_importances[indices[i]]*100))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import pandas as pd

# normalize TF-IDF embeddings
tfidf_embeddings_normalized = normalize(X_tfidf)

# perform k-means clustering
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=1)
kmeans.fit(tfidf_embeddings_normalized)

# get feature importances and vocabulary
feature_importances = kmeans.cluster_centers_
vocabulary = vectorizer_tfidf.vocabulary_

# print the top 5 most important features of each cluster
print("Top n most important features of each cluster:")
for i in range(num_clusters):
    indices = np.argsort(feature_importances[i])[::-1][:10]
    features = [list(vocabulary.keys())[list(vocabulary.values()).index(index)] for index in indices]
    print("Cluster {}: {}".format(i+1, features))


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

cosine_similarities = cosine_similarity(tfidf_embeddings_normalized[750].reshape(1, -1), tfidf_embeddings_normalized).reshape(-1)
df['similarities'] = cosine_similarities
grouped_similarities = df.groupby('clase')['similarities'].apply(list)

# Determine the common range for the x-axis
min_value = df['similarities'].min()
max_value = df['similarities'].max()
bins = 100
common_range = (min_value, max_value)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
axes = axes.flatten()

for i, (class_name, similarities) in enumerate(grouped_similarities.items()):
    axes[i].hist(similarities, bins=bins, range=common_range, alpha=0.5, label=class_name, color=f"C{i}")
    axes[i].set_title(f'Cosine Similarities for Class: {class_name}')
    axes[i].set_xlabel('Cosine Similarity')
    axes[i].set_ylabel('Frequency')

fig.tight_layout()
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import pandas as pd
import umap.umap_ as umap
import matplotlib.pyplot as plt

# perform UMAP dimensionality reduction on all data points
umap_embeddings = umap.UMAP().fit_transform(tfidf_embeddings_normalized)

# assign a color to each cluster label
cmap = plt.get_cmap('viridis')
colors = [cmap(i) for i in np.linspace(0, 1, num_clusters)]

# plot all data points with different colors and labels for each cluster
for i in range(num_clusters):
    # get the indices of data points in this cluster
    indices = kmeans.labels_ == i
    # plot the data points with a different color and label name for this cluster
    plt.scatter(umap_embeddings[indices, 0], umap_embeddings[indices, 1], c=[colors[i]], label=f'Cluster {i}')
    
# add legend to the plot
plt.legend()

# show the plot
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import pandas as pd

df_subclusters = df_politica[['content', 'clase']].dropna()

# Train the TfidfVectorizer and transform the texts
X_tfidf = vectorizer_tfidf.fit_transform(df_subclusters['content']).todense()
y = df['clase']

# normalize TF-IDF embeddings
tfidf_embeddings_normalized = normalize(X_tfidf)

# perform k-means clustering
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=1)
kmeans.fit(tfidf_embeddings_normalized)

# get feature importances and vocabulary
feature_importances = kmeans.cluster_centers_
vocabulary = vectorizer_tfidf.vocabulary_

# print the top 5 most important features of each cluster
print("Top n most important features of each cluster:")
for i in range(num_clusters):
    indices = np.argsort(feature_importances[i])[::-1][:10]
    features = [list(vocabulary.keys())[list(vocabulary.values()).index(index)] for index in indices]
    print("Cluster {}: {}".format(i+1, features))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import pandas as pd

df_subclusters = df_deportes[['content', 'clase']].dropna()

# Train the TfidfVectorizer and transform the texts
X_tfidf = vectorizer_tfidf.fit_transform(df_subclusters['content']).todense()
y = df['clase']

# normalize TF-IDF embeddings
tfidf_embeddings_normalized = normalize(X_tfidf)

# perform k-means clustering
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=1)
kmeans.fit(tfidf_embeddings_normalized)

# get feature importances and vocabulary
feature_importances = kmeans.cluster_centers_
vocabulary = vectorizer_tfidf.vocabulary_

# print the top 5 most important features of each cluster
print("Top n most important features of each cluster:")
for i in range(num_clusters):
    indices = np.argsort(feature_importances[i])[::-1][:10]
    features = [list(vocabulary.keys())[list(vocabulary.values()).index(index)] for index in indices]
    print("Cluster {}: {}".format(i+1, features))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import pandas as pd

df_subclusters = df_economia[['content', 'clase']].dropna()

# Train the TfidfVectorizer and transform the texts
X_tfidf = vectorizer_tfidf.fit_transform(df_subclusters['content']).todense()
y = df['clase']

# normalize TF-IDF embeddings
tfidf_embeddings_normalized = normalize(X_tfidf)

# perform k-means clustering
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=1)
kmeans.fit(tfidf_embeddings_normalized)

# get feature importances and vocabulary
feature_importances = kmeans.cluster_centers_
vocabulary = vectorizer_tfidf.vocabulary_

# print the top 5 most important features of each cluster
print("Top n most important features of each cluster:")
for i in range(num_clusters):
    indices = np.argsort(feature_importances[i])[::-1][:10]
    features = [list(vocabulary.keys())[list(vocabulary.values()).index(index)] for index in indices]
    print("Cluster {}: {}".format(i+1, features))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Download the NLTK Spanish stopwords if you haven't already
spanish_stopwords = nltk.corpus.stopwords.words('spanish')

# Initialize the TfidfVectorizer
vectorizer_tfidf = TfidfVectorizer(max_features=2000, stop_words=spanish_stopwords, ngram_range=(1,3), lowercase=True)

df_codo = df[['content', 'clase']].dropna()

# Train the TfidfVectorizer and transform the texts
X_tfidf = vectorizer_tfidf.fit_transform(df_codo['content']).todense()

# normalize TF-IDF embeddings
tfidf_embeddings_normalized = normalize(X_tfidf)

# elbow plot
inertias = []
for i in range(2, 50):
    kmeans = KMeans(n_clusters=i, random_state=1)
    kmeans.fit(tfidf_embeddings_normalized)
    inertias.append(kmeans.inertia_)
    
plt.plot(range(2, 50), inertias)
plt.title('Elbow Plot')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
# perform k-means clustering with optimal number of clusters
num_clusters = 6 # choose the number of clusters based on the elbow plot
kmeans = KMeans(n_clusters=num_clusters, random_state=1)
kmeans.fit(tfidf_embeddings_normalized)

# get feature importances and vocabulary
feature_importances = kmeans.cluster_centers_
vocabulary = vectorizer_tfidf.vocabulary_

# print the top 5 most important features of each cluster
print("Top n most important features of each cluster:")
for i in range(num_clusters):
    indices = np.argsort(feature_importances[i])[::-1][:10]
    features = [list(vocabulary.keys())[list(vocabulary.values()).index(index)] for index in indices]
    print("Cluster {}: {}".format(i+1, features))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np

# Train the TfidfVectorizer and transform the texts
spanish_stopwords = nltk.corpus.stopwords.words('spanish')

vectorizer_tfidf = TfidfVectorizer(max_features=2000, stop_words=spanish_stopwords)
X_tfidf = vectorizer_tfidf.fit_transform(df['content']).todense()

# Normalize the embeddings
X_tfidf_normalized = normalize(X_tfidf)

# Train a GMM model with a fixed number of clusters
num_clusters = 4
gmm = GaussianMixture(n_components=num_clusters, covariance_type='diag', random_state=0)
gmm.fit(X_tfidf_normalized)

# Predict the cluster assignments for each text
cluster_assignments = gmm.predict(X_tfidf_normalized)

# Calculate the average TF-IDF weights for each word in each cluster
feature_weights = gmm.weights_[:, np.newaxis] * gmm.means_

# Get the vocabulary from the TfidfVectorizer
vocabulary = vectorizer_tfidf.vocabulary_

# Print the top 10 most important words for each cluster
print("Top 10 most important words for each cluster:")
for i in range(num_clusters):
    indices = np.argsort(feature_weights[i])[::-1][:10]
    features = [list(vocabulary.keys())[list(vocabulary.values()).index(index)] for index in indices]
    print("Cluster {}: {}".format(i+1, features))

In [ ]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

df_educacion = pd.read_csv("noticias_educacion_sample.csv")
df_educacion['clase'] = 0
df_politica = pd.read_csv("noticias_politica_sample.csv")
df_politica['clase'] = 1
df_deportes = pd.read_csv("noticias_deportes_sample.csv")
df_deportes['clase'] = 2
df_economia = pd.read_csv("noticias_economia_sample.csv")
df_economia['clase'] = 3
df = pd.concat([df_educacion, df_politica, df_deportes, df_economia]).dropna().reset_index()

# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to get the compound sentiment score
def get_sentiment(text):
    if isinstance(text, str):
        sentiment = analyzer.polarity_scores(text)
        return sentiment['compound']
    else:
        return None

# Apply the sentiment analysis to the 'content' column and store the results in a new column called 'sentiment'
df.loc[0:1000, 'sentiment'] = df.loc[0:1000, 'content'].apply(get_sentiment)


In [ ]:
# Display the DataFrame with the new 'sentiment' column
df_positive = df[df['sentiment'].astype(float)>0.95].reset_index()
for i in range(10):
    print(df_positive.loc[i, 'headline'])

In [ ]:
# Display the DataFrame with the new 'sentiment' column
df_negative = df[df['sentiment'].astype(float)<-0.99].reset_index()
for i in range(10):
    print(df_negative.loc[i, 'headline'])

In [ ]:
import spacy

# Load the Spanish language model
nlp = spacy.load('es_core_news_lg')

# Process the text with the language model
doc = nlp(df_politica.loc[10, 'content'])

# Print the entities found in the text
for ent in doc.ents:
    if ent.label_ in ['PER', 'LOC']:
        print(ent.text, ent.label_)